## Images

In [9]:
import os
import json
import requests

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import cv2

import tensorflow as tf
%matplotlib inline

In [14]:
tf.__version__

'2.1.0'

In [ ]:
tf.__version__

In [ ]:
def show_img(src, figsize=(10,10)):
    '''
    Описание:
        Вывод картинки src.
    Вход:
        src - исходный массив np
        figsize - размер выводимой картинки
    Выход:
        -
    ''' 
    plt.figure(figsize=figsize)
    plt.imshow(src)
    plt.show()

def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou
def get_stop_line_box():
    # return (0, 348, 408, 700)
    # return (348,0, 372, 408)
    return (348,0, 348+372, 408)

In [ ]:
# Params 
path_to_saved_model = "models\\my_model_ssd_mobilenet_v2_fpnlite\\1"
path_to_images = 'data\\photo'
stop_line_box = get_stop_line_box()
iou_thresh = 0.1

USE_TF_SERVING = False
detection_model = None
if not USE_TF_SERVING:
    detection_model = tf.saved_model.load(path_to_saved_model)

In [ ]:
file_names = ['file503.png', 'file504.png','file529.png','file570.png','file575.png','file586.png']
stop_line_box = get_stop_line_box()

for image_name in file_names:
    # Read image
    image_path = os.path.join(path_to_images, image_name)
    image = Image.open(image_path)

    # Requests to models server 
    image_np = np.array(image)
    w_image, h_image, _ = image_np.shape
    
    if USE_TF_SERVING:
        payload = {"instances": [image_np.tolist()]}
        res = requests.post("http://192.168.99.100:8501/v1/models/detection:predict", json=payload)
        detections = res.json()["predictions"][0]
    else:
        input_tensor = tf.convert_to_tensor(image_np)
        input_tensor = input_tensor[tf.newaxis, ...]
        detections = detection_model(input_tensor)
    
    # Prepare response
    scores = np.array(detections['detection_scores']).squeeze()
    labels = np.array(detections['detection_classes']).squeeze()

    boxes = np.array(detections['detection_boxes']).squeeze()
    boxes = np.array(boxes)
    boxes = boxes*np.array([w_image, h_image, w_image, h_image])
    boxes = boxes.astype('int16')

    # Visualize results
    res_image = image_np.copy()
    score_tresh = 0.30
    for i in range(len(scores)):
        if scores[i] > score_tresh:
            box = boxes[i] 
            box_color = (0,255,0)
            # TODO: ALARM! Update base!
            if  bb_intersection_over_union(box, stop_line_box) > iou_thresh:
                print(bb_intersection_over_union(box, stop_line_box))
                box_color = (255,0,0)
            
            # print box
            cv2.rectangle(res_image, (box[1], box[0]), (box[3], box[2]), box_color)

    cv2.rectangle( 
        res_image, 
        (stop_line_box[1], stop_line_box[0]), 
        (stop_line_box[3], stop_line_box[2]), 
        (255,0,0))
    show_img(res_image)

## Video with write result video

In [ ]:
import os
import json
import requests

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from imutils.video import VideoStream
from imutils.video import FPS
import imutils

import cv2

import tensorflow as tf

%matplotlib inline

In [ ]:
def show_img(src, figsize=(10,10)):
    '''
    Описание:
        Вывод картинки src.
    Вход:
        src - исходный массив np
        figsize - размер выводимой картинки
    Выход:
        -
    ''' 
    plt.figure(figsize=figsize)
    plt.imshow(src)
    plt.show()

def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou
	
def get_stop_line_box():
    return (200, 200, 600, 600)

In [ ]:
path_to_video =  'data\\video'
path_to_saved_model = "models\\my_model_ssd_mobilenet_v2_fpnlite\\1"
video_filename = 'KIT-AXL-2020-12-08-09-10-01-581.mp4'
video_out_file_path = 'data\\video\\{}_out.avi'.format(video_filename)

tf_serving_url ="http://192.168.99.100:8501/v1/models/detection:predict"

seconds = 2
iou_thresh = 0.00
score_tresh = 0.1
USE_TF_SERVING = False

stop_line_box = get_stop_line_box()

detection_model = None
if not USE_TF_SERVING:
    detection_model = tf.saved_model.load(path_to_saved_model)

In [ ]:
print("[INFO] starting video stream...")
vs = cv2.VideoCapture(os.path.join(path_to_video, video_filename))

fps = vs.get(cv2.CAP_PROP_FPS) # Gets the frames per second
multiplier = int(fps * seconds)

frame_count = int(vs.get(cv2.CAP_PROP_FRAME_COUNT))
frameId = 0

video_writer = cv2.VideoWriter()
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
# loop over the frames from the video stream
while True:
    # read frame
    frame = vs.read()
    frameId = int(round(vs.get(1)))    
    # check to see if we have reached the end of the stream
    if frameId >= frame_count:
        break

    frame = frame[1]
    if frameId == 1: 
        video_writer = cv2.VideoWriter(video_out_file_path, fourcc, 1.0, (frame.shape[1],frame.shape[0]))

    # frame = imutils.resize(frame, width=460)

    if frameId % multiplier == 0:
        # # Detect persons
        image_np = frame.copy()
        w_image, h_image, _ = image_np.shape
        
        if USE_TF_SERVING:
            payload = {"instances": [image_np.tolist()]}
            res = requests.post("http://192.168.99.100:8501/v1/models/detection:predict", json=payload)
            detections = res.json()["predictions"][0]
        else:
            input_tensor = tf.convert_to_tensor(image_np)
            input_tensor = input_tensor[tf.newaxis, ...]
            detections = detection_model(input_tensor)
        
        # Prepare response
        scores = np.array(detections['detection_scores']).squeeze()
        labels = np.array(detections['detection_classes']).squeeze()

        boxes = np.array(detections['detection_boxes']).squeeze()
        boxes = np.array(boxes)
        boxes = boxes*np.array([w_image, h_image, w_image, h_image])
        boxes = boxes.astype('int16')

        # Visualize results
        res_image = image_np.copy()
        
        for i in range(len(scores)):
            if scores[i] > score_tresh:
                box = boxes[i] 
                box_color = (0,255,0)
                # TODO: ALARM! Update base!
                if  bb_intersection_over_union(box, stop_line_box) > iou_thresh:
                    # print(bb_intersection_over_union(box, stop_line_box))
                    box_color = (0,0,255)
                
                # print box
                cv2.rectangle(res_image, (box[1], box[0]), (box[3], box[2]), box_color)

        cv2.rectangle( 
            res_image, 
            (stop_line_box[1], stop_line_box[0]), 
            (stop_line_box[3], stop_line_box[2]), 
            (0,0,255))

        cv2.imshow("Frame", res_image)
        video_writer.write(res_image)

    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
    print('\r{}/{}'.format(frameId,frame_count),end='')

# do a bit of cleanup
vs.release()
video_writer.release()
cv2.destroyAllWindows()

## Video with ROI

In [2]:
import os
import datetime
import json
import requests

from imutils.video import VideoStream
from imutils.video import FPS
import imutils
from PIL import Image
import cv2

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def bb_intersection_over_union(boxA, boxB):
    '''
    IOU for person detection in alarm box.
    '''
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [ ]:
# parameters
path_to_video =  'data\\video'
video_filename = 'KIT-AXL-2020-12-08-09-10-01-581.mp4'

# config
path_to_saved_model = "models\\my_model_ssd_mobilenet_v2_fpnlite\\1"
tf_serving_url ="http://192.168.99.100:8501/v1/models/detection:predict"
data_base_path = 'db'
seconds = 2
iou_thresh = 0.00
score_tresh = 0.12
USE_TF_SERVING = False
WRITE_RESULT_VIDEO = True
stop_line_box = (0,0,100,100)

# Init steps
if WRITE_RESULT_VIDEO:
    video_out_file_path = os.path.join(path_to_video, '{}_out.avi'.format(video_filename))
detection_model = None
if not USE_TF_SERVING:
    detection_model = tf.saved_model.load(path_to_saved_model)

In [ ]:
print("[INFO] starting video stream...")
# Open VideoCapture
path_to_videofile = os.path.join(path_to_video, video_filename)
vs = cv2.VideoCapture(path_to_videofile)
# Open VideoWriter
if WRITE_RESULT_VIDEO:
    # video_writer = cv2.VideoWriter()
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')

# Prepare for frames loop info
fps = vs.get(cv2.CAP_PROP_FPS)
multiplier = int(fps * seconds)
frame_count = int(vs.get(cv2.CAP_PROP_FRAME_COUNT))
frameId = 0

alarm_time_on_video = []
# loop over the frames from the video stream
while True:
    # read frame
    frame = vs.read()
    frameId = int(round(vs.get(1)))    
    # check to see if we have reached the end of the stream
    if frameId >= frame_count:
        break

    frame = frame[1]
    if frameId == 1: 
        if WRITE_RESULT_VIDEO:
            video_writer = cv2.VideoWriter(video_out_file_path, fourcc, 1.0, (frame.shape[1],frame.shape[0]))
        # Get ROI
        stop_line_box = cv2.selectROI('Select box', frame, False, False)
        stop_line_box = [
            stop_line_box[1], 
            stop_line_box[0],
            stop_line_box[3] + stop_line_box[1],
            stop_line_box[2] + stop_line_box[0],
            ]
        cv2.destroyWindow('Select box') 
        print(stop_line_box)

    if frameId % multiplier == 0:
        # Detect persons
        image_np = frame.copy()
        w_image, h_image, _ = image_np.shape
        
        if USE_TF_SERVING:
            payload = {"instances": [image_np.tolist()]}
            res = requests.post("http://192.168.99.100:8501/v1/models/detection:predict", json=payload)
            detections = res.json()["predictions"][0]
        else:
            input_tensor = tf.convert_to_tensor(image_np)
            input_tensor = input_tensor[tf.newaxis, ...]
            detections = detection_model(input_tensor)
        
        # Prepare response
        scores = np.array(detections['detection_scores']).squeeze()
        labels = np.array(detections['detection_classes']).squeeze()

        boxes = np.array(detections['detection_boxes']).squeeze()
        boxes = np.array(boxes)
        boxes = boxes*np.array([w_image, h_image, w_image, h_image])
        boxes = boxes.astype('int16')

        # Visualize results
        res_image = image_np.copy()
        
        for i in range(len(scores)):
            if scores[i] > score_tresh:
                box = boxes[i] 
                box_color = (0,255,0)
                # TODO: ALARM! Update base!
                if  bb_intersection_over_union(box, stop_line_box) > iou_thresh:
                    # print(bb_intersection_over_union(box, stop_line_box))
                    box_color = (0,0,255)
                    alarm_time_on_video.append(str(datetime.timedelta(seconds=int(seconds*frameId/multiplier))))
                    print('-->',str(datetime.timedelta(seconds=int(frameId/multiplier))))
                
                # print box
                cv2.rectangle(res_image, (box[1], box[0]), (box[3], box[2]), box_color)

        cv2.rectangle( 
            res_image, 
            (stop_line_box[1], stop_line_box[0]), 
            (stop_line_box[3], stop_line_box[2]), 
            (0,0,255))

        cv2.imshow("Frame", res_image)
        if WRITE_RESULT_VIDEO:
            video_writer.write(res_image)

    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
    print('\r{}/{}'.format(frameId,frame_count),end='')

# Do a bit of cleanup
vs.release()
if WRITE_RESULT_VIDEO:
    video_writer.release()
cv2.destroyAllWindows()

# Update db
result_json = {
    'filename':video_filename,
    'w_h':[w_image, h_image],
    'alarm_box':stop_line_box,
    'alarm_time':alarm_time_on_video
}

result_json_filename = '{}_db.json'.format(video_filename)
with open(os.path.join(data_base_path, result_json_filename), 'w') as f:
    json.dump(result_json, f)

print("[INFO] Done!")